<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий на hh.ru
   

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go

# Исследование структуры данных

1. Прочитайте данные с помощью библиотеки Pandas. Совет: перед чтением обратите внимание на разделитель внутри файла. 

In [ ]:
# Прочитаем таблицу из csv файла
hh_data = pd.read_csv('dst-3.0_16_1_hh_database.csv', sep=';')

# Задание 2.1
print(f'Размерность таблицы: {hh_data.shape}')

2. Выведите несколько первых (последних) строк таблицы, чтобы убедиться в том, что ваши данные не повреждены. Ознакомьтесь с признаками и их структурой.

In [ ]:
display(hh_data.head(2))

3. Выведите основную информацию о числе непустых значений в столбцах и их типах в таблице.

4. Обратите внимание на информацию о числе непустых значений.

In [ ]:
hh_data.info()

5. Выведите основную статистическую информацию о столбцах.


In [ ]:
hh_data.describe()

# Преобразование данных

1. Начнем с простого - с признака **"Образование и ВУЗ"**. Его текущий формат это: **<Уровень образования год выпуска ВУЗ специальность...>**. Например:
* Высшее образование 2016 Московский авиационный институт (национальный исследовательский университет)...
* Неоконченное высшее образование 2000  Балтийская государственная академия рыбопромыслового флота…
Нас будет интересовать только уровень образования.

Создайте с помощью функции-преобразования новый признак **"Образование"**, который должен иметь 4 категории: "высшее", "неоконченное высшее", "среднее специальное" и "среднее".

Выполните преобразование, ответьте на контрольные вопросы и удалите признак "Образование и ВУЗ".

Совет: обратите внимание на структуру текста в столбце **"Образование и ВУЗ"**. Гарантируется, что текущий уровень образования соискателя всегда находится в первых 2ух слов и начинается с заглавной буквы. Воспользуйтесь этим.

*Совет: проверяйте полученные категории, например, с помощью метода unique()*


In [ ]:
# Скопируем данные во избежание необратимых изменеий
hh_copy = hh_data.copy()

def get_education_level(string):
    """Функция позволяет получить уровень образования из признака "Образование и ВУЗ"

    Args:
        string (string): строка, содержащая информацию об образования 
        
    Returns:
        string: уровень образования, один из 4 следующих: "высшее", "неоконченное высшее", "среднее специальное", "среднее"
    """    
    string = string.split()[:2]
    
    if string[0] == 'Неоконченное' or string[1] == 'специальное':
        return ' '.join(string).lower()
    else:
        return str(string[0]).lower()

# Создадим новый признак 'Образование' и удалим 'Образование и ВУЗ'
hh_copy['Образование'] = hh_copy['Образование и ВУЗ'].apply(get_education_level)
hh_copy = hh_copy.drop(columns='Образование и ВУЗ') 

# задание 3.1
print(f'Количество соискателей со средним уровнем образования: {hh_copy[hh_copy["Образование"] == "среднее"].shape[0]}')

2. Теперь нас интересует столбец **"Пол, возраст"**. Сейчас он представлен в формате **<Пол , возраст , дата рождения >**. Например:
* Мужчина , 39 лет , родился 27 ноября 1979 
* Женщина , 21 год , родилась 13 января 2000
Как вы понимаете, нам необходимо выделить каждый параметр в отдельный столбец.

Создайте два новых признака **"Пол"** и **"Возраст"**. При этом важно учесть:
* Признак пола должен иметь 2 уникальных строковых значения: 'М' - мужчина, 'Ж' - женщина. 
* Признак возраста должен быть представлен целыми числами.

Выполните преобразование, ответьте на контрольные вопросы и удалите признак **"Пол, возраст"** из таблицы.

*Совет: обратите внимание на структуру текста в столбце, в части на то, как разделены параметры пола, возраста и даты рождения между собой - символом ' , '. 
Гарантируется, что структура одинакова для всех строк в таблице. Вы можете воспользоваться этим.*


In [ ]:
# Создадим функции для получения возраста и пола
def get_gender(string):
    """Функция позволяет получить пол из признака "Пол, возраст" 

    Args:
        string (string): строка, содержащая информаю о поле и возрасте

    Returns:
        string: возвращает "М" для мужчин и "Ж" для женщин
    """    
    string = string.split(',')
    if string[0].strip() == 'Мужчина':
        return 'М'
    else:
        return 'Ж'


def get_age(string):
    """Функция определяет возраст из признака "Пол, возраст" 

    Args:
        string (string): строка, содержащая информаю о поле и возрасте

    Returns:
        int: возраст
    """    
    string = string.split(',')
    string = str(string[1]).split()[0]
    return int(string)

# Создадим новые признаки для пола и возраста, удалим признак "Пол, возраст"
hh_copy['Пол'] = hh_copy["Пол, возраст"].apply(get_gender)
hh_copy['Возраст'] = hh_copy["Пол, возраст"].apply(get_age)
hh_copy = hh_copy.drop(columns="Пол, возраст")

# задание 3.2
print(f'Процент женских резюме: {round(hh_copy["Пол"].value_counts(normalize=True).loc["Ж"]*100, 2)}')
print(f'Средний возраст соискателей: {round(hh_copy["Возраст"].mean(), 1)}')

3. Следующим этапом преобразуем признак **"Опыт работы"**. Его текущий формат - это: **<Опыт работы: n лет m месяцев, периоды работы в различных компаниях…>**. 

Из столбца нам необходимо выделить общий опыт работы соискателя в месяцах, новый признак назовем "Опыт работы (месяц)"

Для начала обсудим условия решения задачи:
* Во-первых, в данном признаке есть пропуски. Условимся, что если мы встречаем пропуск, оставляем его как есть (функция-преобразование возвращает NaN)
* Во-вторых, в данном признаке есть скрытые пропуски. Для некоторых соискателей в столбце стоит значения "Не указано". Их тоже обозначим как NaN (функция-преобразование возвращает NaN)
* В-третьих, нас не интересует информация, которая описывается после указания опыта работы (периоды работы в различных компаниях)
* В-четвертых, у нас есть проблема: опыт работы может быть представлен только в годах или только месяцах. Например, можно встретить следующие варианты:
    * Опыт работы 3 года 2 месяца…
    * Опыт работы 4 года…
    * Опыт работы 11 месяцев…
    * Учитывайте эту особенность в вашем коде

Учитывайте эту особенность в вашем коде

В результате преобразования у вас должен получиться столбец, содержащий информацию о том, сколько месяцев проработал соискатель.
Выполните преобразование, ответьте на контрольные вопросы и удалите столбец **"Опыт работы"** из таблицы.


In [ ]:
# Создадим функцию для получения опыта работы в месяцах
def get_months(string):
    """Функция определяет опыт работы в месяцах 

    Args:
        string (string): строка, содержащая информацию о стаже и местах работы

    Returns:
        int: опыт работы в месяцах
    """    
    if string == 'Не указано' or string is np.nan or str(string) == 'nan':
        return np.nan

    year_words = ['лет', 'года', 'год']
    month_words = ['месяц', 'месяца', 'месяцев']
    
    string = str(string).split()[:6]

    months = 0
    years = 0

    if string[3] in year_words:
        years += int(string[2])

    if string[3] in month_words:
        months += int(string[2])
    elif string[5] in month_words:
        months += int(string[4])

    return (years * 12) + months

# Создадим новый признак 'Опыт работы (месяц)' и удалим признак 'Опыт работы'
hh_copy['Опыт работы (месяц)'] = hh_copy['Опыт работы'].apply(get_months)
hh_copy = hh_copy.drop(columns='Опыт работы')

# Задание 3.3
print(f"Медианный опыт работы в месяцах: {hh_copy['Опыт работы (месяц)'].median()}")

4. Хорошо идем! Следующий на очереди признак "Город, переезд, командировки". Информация в нем представлена в следующем виде: **<Город , (метро) , готовность к переезду (города для переезда) , готовность к командировкам>**. В скобках указаны необязательные параметры строки. Например, можно встретить следующие варианты:

* Москва , не готов к переезду , готов к командировкам
* Москва , м. Беломорская , не готов к переезду, не готов к командировкам
* Воронеж , готов к переезду (Сочи, Москва, Санкт-Петербург) , готов к командировкам

Создадим отдельные признаки **"Город"**, **"Готовность к переезду"**, **"Готовность к командировкам"**. При этом важно учесть:

* Признак **"Город"** должен содержать только 4 категории: "Москва", "Санкт-Петербург" и "город-миллионник" (их список ниже), остальные обозначьте как "другие".

    Список городов-миллионников:
    
   <code>million_cities = ['Новосибирск', 'Екатеринбург','Нижний Новгород','Казань', 'Челябинск','Омск', 'Самара', 'Ростов-на-Дону', 'Уфа', 'Красноярск', 'Пермь', 'Воронеж','Волгоград']
    </code>
    Инфорация о метро, рядом с которым проживает соискатель нас не интересует.
* Признак **"Готовность к переезду"** должен иметь два возможных варианта: True или False. Обратите внимание, что возможны несколько вариантов описания готовности к переезду в признаке "Город, переезд, командировки". Например:
    * … , готов к переезду , …
    * … , не готова к переезду , …
    * … , готова к переезду (Москва, Санкт-Петербург, Ростов-на-Дону)
    * … , хочу переехать (США) , …
    
    Нас интересует только сам факт возможности или желания переезда.
* Признак **"Готовность к командировкам"** должен иметь два возможных варианта: True или False. Обратите внимание, что возможны несколько вариантов описания готовности к командировкам в признаке "Город, переезд, командировки". Например:
    * … , готов к командировкам , … 
    * … , готова к редким командировкам , …
    * … , не готов к командировкам , …
    
    Нас интересует только сам факт готовности к командировке.
    
    Еще один важный факт: при выгрузки данных у некоторых соискателей "потерялась" информация о готовности к командировкам. Давайте по умолчанию будем считать, что такие соискатели не готовы к командировкам.
    
Выполните преобразования и удалите столбец **"Город, переезд, командировки"** из таблицы.

*Совет: обратите внимание на то, что структура текста может меняться в зависимости от указания ближайшего метро. Учите это, если будете использовать порядок слов в своей программе.*


In [ ]:
# Создадим функции для определения города и статуса готовности к переезду и командировкам
def get_city(string):
    """Функция определяет город проживания из строки из признака "Город, переезд, командировки"

    Args:
        string (string): строка, содержащая инормацию из признака "Город, переезд, командировки"

    Returns:
        string: Строка с одним из 4 значений: "Москва", "Санкт-Петербург", "город миллионник" или "другие"
    """ 
    million_cities = ['Новосибирск', 'Екатеринбург','Нижний Новгород','Казань', 
                  'Челябинск','Омск', 'Самара', 'Ростов-на-Дону', 'Уфа', 'Красноярск', 'Пермь', 'Воронеж','Волгоград']
    string = string.split(sep=',')[0].strip()
    if string in ['Москва', 'Санкт-Петербург']:
        return string
    elif string in million_cities:
        return 'город-миллионник'
    else:
        return 'другие'


def get_relocate(string):
    """Функция определяет готов ли человек на переезд 

    Args:
        string (string): строка, содержащая инормацию из признака "Город, переезд, командировки"

    Returns:
        bool: True - готов к переезду, False - не готов
    """    
    if ('не готов к переезду' in string) or ('не готова к переезду' in string):
        return False
    else:
        return True


def get_business_trip(string):
    """Функция определяет готов ли человек на командировки

    Args:
        string (string): строка, содержащая инормацию из признака "Город, переезд, командировки"

    Returns:
        bool: True - готов к командировкам, False - не готов
    """    
    if ('командировка' in string):
        if ('не готов к командировкам' in string) or ('не готова к командировкам' in string):
            return False
        else:
            return True
    else:
        return False

# Создадим новые признаки и удалим "Город, переезд, командировки"
hh_copy["Город"] = hh_copy["Город, переезд, командировки"].apply(get_city)
hh_copy["Готовность к переезду"] = hh_copy["Город, переезд, командировки"].apply(get_relocate)
hh_copy["Готовность к командировкам"] = hh_copy["Город, переезд, командировки"].apply(get_business_trip)
hh_copy = hh_copy.drop(columns="Город, переезд, командировки")

# Задание 3.4
print(f'В Санкт-Петербурге проживает {round(hh_copy["Город"].value_counts(normalize=True).loc["Санкт-Петербург"]*100, 2)}% сосикателей')

mask = (hh_copy['Готовность к переезду'] == True) & (hh_copy["Готовность к командировкам"] == True)
print(f'И к переезду и к командировкам готовы {round(hh_copy[mask].shape[0] / (hh_copy.shape[0] / 100))}% соискателей')

5. Рассмотрим поближе признаки **"Занятость"** и **"График"**. Сейчас признаки представляют собой набор категорий желаемой занятости (полная занятость, частичная занятость, проектная работа, волонтерство, стажировка) и желаемого графика работы (полный день, сменный график, гибкий график, удаленная работа, вахтовый метод).
На сайте hh.ru соискатель может указывать различные комбинации данных категорий, например:
* полная занятость, частичная занятость
* частичная занятость, проектная работа, волонтерство
* полный день, удаленная работа
* вахтовый метод, гибкий график, удаленная работа, полная занятость

Такой вариант признаков имеет множество различных комбинаций, а значит множество уникальных значений, что мешает анализу. Нужно это исправить!

Давайте создадим признаки-мигалки для каждой категории: если категория присутствует в списке желаемых соискателем, то в столбце на месте строки рассматриваемого соискателя ставится True, иначе - False.

Такой метод преобразования категориальных признаков называется One Hot Encoding и его схема представлена на рисунке ниже:
<img src=https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/ohe.jpg>
Выполните данное преобразование для признаков "Занятость" и "График", ответьте на контрольные вопросы, после чего удалите их из таблицы

In [ ]:
#Используем цикл for для создания признаков-мигалок
employments = ['полная занятость', 'частичная занятость', 'проектная работа', 'волонтерство', 'стажировка']
schedules = ['полный день', 'сменный график', 'гибкий график', 'удаленная работа', 'вахтовый метод']
for employment,schedule in zip(employments, schedules):
    hh_copy[employment] = hh_copy['Занятость'].apply(lambda x: employment in x)
    hh_copy[schedule] =  hh_copy['График'].apply(lambda x: schedule in x)

# Удалим признаки 'Занятость' и 'График'
hh_copy = hh_copy.drop(columns=['Занятость', 'График'])

# Задание 3.5
mask = (hh_copy['проектная работа'] == True) & (hh_copy['волонтерство'] == True)
print(f'И проектную работу и волонтерство ищет {hh_copy[mask].shape[0]} людей')

mask = (hh_copy['вахтовый метод'] == True) & (hh_copy['гибкий график'] == True)
print(f'Вахтовым методом и с гибким графиком хотят работать {hh_copy[mask].shape[0]} людей')


6. (2 балла) Наконец, мы добрались до самого главного и самого важного - признака заработной платы **"ЗП"**. 
В чем наша беда? В том, что помимо желаемой заработной платы соискатель указывает валюту, в которой он бы хотел ее получать, например:
* 30000 руб.
* 50000 грн.
* 550 USD

Нам бы хотелось видеть заработную плату в единой валюте, например, в рублях. Возникает вопрос, а где взять курс валют по отношению к рублю?

На самом деле язык Python имеет в арсенале огромное количество возможностей получения данной информации, от обращения к API Центробанка, до использования специальных библиотек, например pycbrf. Однако, это не тема нашего проекта.

Поэтому мы пойдем в лоб: обратимся к специальным интернет-ресурсам для получения данных о курсе в виде текстовых файлов. Например, MDF.RU, данный ресурс позволяет удобно экспортировать данные о курсах различных валют и акций за указанные периоды в виде csv файлов. Мы уже сделали выгрузку курсов валют, которые встречаются в наших данных за период с 29.12.2017 по 05.12.2019. Скачать ее вы можете **на платформе**

Создайте новый DataFrame из полученного файла. В полученной таблице нас будут интересовать столбцы:
* "currency" - наименование валюты в ISO кодировке,
* "date" - дата, 
* "proportion" - пропорция, 
* "close" - цена закрытия (последний зафиксированный курс валюты на указанный день).


Перед вами таблица соответствия наименований иностранных валют в наших данных и их общепринятых сокращений, которые представлены в нашем файле с курсами валют. Пропорция - это число, за сколько единиц валюты указан курс в таблице с курсами. Например, для казахстанского тенге курс на 20.08.2019 составляет 17.197 руб. за 100 тенге, тогда итоговый курс равен - 17.197 / 100 = 0.17197 руб за 1 тенге.
Воспользуйтесь этой информацией в ваших преобразованиях.

<img src=https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/table.jpg>


Осталось только понять, откуда брать дату, по которой определяется курс? А вот же она - в признаке **"Обновление резюме"**, в нем содержится дата и время, когда соискатель выложил текущий вариант своего резюме. Нас интересует только дата, по ней бы и будем сопоставлять курсы валют.

Теперь у нас есть вся необходимая информация для того, чтобы создать признак "ЗП (руб)" - заработная плата в рублях.

После ответа на контрольные вопросы удалите исходный столбец заработной платы "ЗП" и все промежуточные столбцы, если вы их создавали.

Итак, давайте обсудим возможный алгоритм преобразования: 
1. Перевести признак "Обновление резюме" из таблицы с резюме в формат datetime и достать из него дату. В тот же формат привести признак "date" из таблицы с валютами.
2. Выделить из столбца "ЗП" сумму желаемой заработной платы и наименование валюты, в которой она исчисляется. Наименование валюты перевести в стандарт ISO согласно с таблицей выше.
3. Присоединить к таблице с резюме таблицу с курсами по столбцам с датой и названием валюты (подумайте, какой тип объединения надо выбрать, чтобы в таблице с резюме сохранились данные о заработной плате, изначально представленной в рублях). Значение close для рубля заполнить единицей 1 (курс рубля самого к себе)
4. Умножить сумму желаемой заработной платы на присоединенный курс валюты (close) и разделить на пропорцию (обратите внимание на пропуски после объединения в этих столбцах), результат занести в новый столбец "ЗП (руб)".


In [ ]:
# Считаем таблицу из csv файла
# И переведем признаки 'date' и 'Обновление резюме' в формат datetime
exchange_rates = pd.read_csv('ExchangeRates.csv')
exchange_rates['date'] = pd.to_datetime(exchange_rates['date'], dayfirst=True).dt.date
hh_copy['Обновление резюме'] = pd.to_datetime(hh_copy['Обновление резюме'], dayfirst=True).dt.date

# Создадим словарь для перевода курса валюты в стандарт ISO
currency_dict = {
    'грн.':'UAH',
    'бел.руб.':'BYN',
    'сум':'UZS',
    'USD':'USD',
    'EUR':'EUR',   
    'KGS':'KGS',   
    'AZN':'AZN',   
    'KZT':'KZT'   
}

# Создадим функцию для перевода курса валюты в стандарт ISO
def get_currency(string):
    """Функция определяет курс валюты в ISO формате из признака 'ЗП'

    Args:
        string (string): строка с информацией о сумме зарплаты и валюте

    Returns:
        string: курс валюты по стандарту ISO, для рубля 'rub'
    """    
    string = string.split()[-1]
    if string == 'руб.':
        return 'rub'
    else:
        return currency_dict[string]
    
# Создадим новые признаки
hh_copy['валюта']= hh_copy['ЗП'].apply(get_currency)
hh_copy['salary'] = hh_copy['ЗП'].apply(lambda x: int(x.split()[0]))

# Объединим таблицы 
hh_copy = hh_copy.merge(
    right=exchange_rates,
    how='left',
    left_on=['Обновление резюме', 'валюта'],
    right_on = ['date', 'currency']
    )

# Заполним пропуски в 'proportion' и 'close' единицами
hh_copy['proportion'] = hh_copy['proportion'].fillna(1)
hh_copy['close'] = hh_copy['close'].fillna(1)

# Создадим признак 'ЗП (руб)' и удалим те, которые больше не нужны
hh_copy['ЗП (руб)'] = hh_copy['salary'] * hh_copy['close'] / hh_copy['proportion']
hh_copy = hh_copy.drop(columns=['proportion', 'vol', 'close', 'time', 'date', 'per', 'currency', 'salary', 'валюта', 'ЗП'])

# Задание 3.6
(f'Медианная зарплата в рублях: {hh_copy["ЗП (руб)"].median()}')

# Исследование зависимостей в данных

1. Постройте распределение признака **"Возраст"**. Опишите распределение, отвечая на следующие вопросы: чему равна мода распределения, каковы предельные значения признака, в каком примерном интервале находится возраст большинства соискателей? Есть ли аномалии для признака возраста, какие значения вы бы причислили к их числу?
*Совет: постройте гистограмму и коробчатую диаграмму рядом.*

In [ ]:
# Создадим функцию для поиска аномальных значений по методу z-отклонения, она потребуется далее
def outliers_z_score(data, feature, log_scale=False, left=3, right=3):
    """Функция ищет аномальные значений по методу z-отклонения

    Args:
        data (DataFrame): Датафрейм, в котором будет вестись поиск аномалий.
        feature (Series): Признак, в котором будет вестись поиск аномалий.
        log_scale (bool, optional): Для логнормального распределения необходимо передать True. Изначально стоит False.
        left (int, optional): Количество сигм для нижней границы. Изначально стоит 3.
        right (int, optional): Количество сигм для верхней границы. Изначально стоит 3.

    Returns:
        outliers (DataFrame): Датафрейм, состоящий из найденных аномальных значений
        cleaned (DataFrame): Датафрейм, очищенный от найденных аномальных значений
    """    
    if log_scale:
        x = np.log(data[feature]+1) 
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - left * sigma
    upper_bound = mu + right * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x > lower_bound) & (x < upper_bound)]
    return outliers, cleaned

In [ ]:
# Создадим гистограмму из изначальных данных и из очищенных методом z-отклонения для визуализации аномалий
fig = px.histogram(
    data_frame=hh_copy,
    title = 'Распределение возраста',
    x='Возраст',
    marginal='box'
)
fig.update_layout(title_xanchor='center', title_x=0.5)
fig.show()


outliers, cleaned = outliers_z_score(data=hh_copy, feature='Возраст', log_scale=True)
fig = px.histogram(
    data_frame=cleaned,
    title = 'Распределение возраста, очищенное от выбросов методом сигм',
    x='Возраст',
    marginal='box'
)
fig.update_layout(title_xanchor='center', title_x=0.5)
fig.show()

Видно логнормальное распределение возраста с модой 30 лет. \
Предельные значения признака: 14-100 лет.\
Возраст большинства соискателей находятся в диапозоне 27-36 лет.\
По методу z-отклонения, аномалиями можно считать 16<возраст>62 \
Видно, что большой рост начинается в 23-24 года, что соответствует возрасту окончания ВУЗа. После 33 начинается сильное, но плавное падение. Возможно, это связанно с тем, что люди этого возраста чаще уже являются состоявшимися специалистами и они могут найти новую работу, используя свои личные связи 

2. Постройте распределение признака **"Опыт работы (месяц)"**. Опишите данное распределение, отвечая на следующие вопросы: чему равна мода распределения, каковы предельные значения признака, в каком примерном интервале находится опыт работы большинства соискателей? Есть ли аномалии для признака опыта работы, какие значения вы бы причислили к их числу?
*Совет: постройте гистограмму и коробчатую диаграмму рядом.*

In [ ]:
# Создадим гистограмму из изначальных данных и из очищенных методом z-отклонения для визуализации аномалий
fig = px.histogram(
    data_frame=hh_copy,
    title = 'Распределение опыта работы в месяцах',
    x='Опыт работы (месяц)',
    marginal='box'
)
fig.update_layout(title_xanchor='center', title_x=0.5)
fig.show()


outliers, cleaned = outliers_z_score(data=hh_copy, feature='Опыт работы (месяц)', log_scale=True)
fig = px.histogram(
    data_frame=cleaned,
    title = 'Распределение опыта работы в месяцах, очищенное от выбросов методом сигм',
    x='Опыт работы (месяц)',
    marginal='box'
)
fig.update_layout(title_xanchor='center', title_x=0.5)
fig.show()

Видно логнормальное распределение возраста с модой 80-84 месяца. \
Предельные значения признака: 1-1188 месяцев. \
Опыт большинства соискателей находятся в диапозоне 57-154 месяца. \
Очевидный выброс с показателем 1188 месяцев(опыт работы 99 лет). По методому z-отклонения, аномалиями можно назвать значения меньше 6 месяцев и больше 663. Но после очистки все ещё виден длинный хвост, возможно стоит пересмотреть параметры для поиска аномалий \
Видимо, самым распространенным типом вариантом соискателя ялвяется человек, который получил высшее образование и отработал 7 лет(мода опыта работы ~7 лет + 23 года соответствует моде возраста соискателя) 


3. Постройте распределение признака **"ЗП (руб)"**. Опишите данное распределение, отвечая на следующие вопросы: каковы предельные значения признака, в каком примерном интервале находится заработная плата большинства соискателей? Есть ли аномалии для признака ЗП? Обратите внимание на гигантские размеры желаемой заработной платы.
*Совет: постройте гистограмму и коробчатую диаграмму рядом.*


In [ ]:
# Создадим гистограмму из изначальных данных и из очищенных методом z-отклонения для визуализации аномалий
fig = px.histogram(
    data_frame=hh_copy,
    title = 'Распределение желаемой зарплаты в рублях',
    x='ЗП (руб)',
    marginal='box'
)
fig.update_layout(title_xanchor='center', title_x=0.5)
fig.show()

# Из-за очень высоких аномалий изначальный график невозможно нормально прочитать, установим ограничение в 550 тысяч 
fig = px.histogram(
    data_frame=hh_copy,
    title = 'Распределение желаемой зарплаты в рублях с ограничением в 550000',
    x='ЗП (руб)',
    range_x= [0,550000],
    nbins = 15000,
    marginal='box'
)
fig.update_layout(title_xanchor='center', title_x=0.5)
fig.show()

outliers, cleaned = outliers_z_score(data=hh_copy, feature='ЗП (руб)', log_scale=True)
fig = px.histogram(
    data_frame=cleaned,
    title = 'Распределение опыта работы в месяцах, очищенное от выбросов методом сигм',
    x='ЗП (руб)',
    marginal='box'
)
fig.update_layout(title_xanchor='center', title_x=0.5)
fig.show()

Логнормальное распределение зарплаты с модой 50 тысяч.\
Видны пики с периодичностью 5000(с увеличением зарплаты периодичность становится = 10000).\
Предельные значения признака: 1-24,3 миллиона(очевидные выбросы).\
Ожидаемая зарплата большинства соискателей находятся в диапозоне 37-95 тысяч.\
По методу z-отклонения, аномалиями можно считать ожидаемую зарплату меньше 6,39 тысяч и выше 517,9 тысяч


4. Постройте диаграмму, которая показывает зависимость **медианной** желаемой заработной платы (**"ЗП (руб)"**) от уровня образования (**"Образование"**). Используйте для диаграммы данные о резюме, где желаемая заработная плата меньше 1 млн рублей.
*Сделайте выводы по представленной диаграмме: для каких уровней образования наблюдаются наибольшие и наименьшие уровни желаемой заработной платы? Как вы считаете, важен ли признак уровня образования при прогнозировании заработной платы?*

In [ ]:
mask = hh_copy['ЗП (руб)'] < 1000000
fig = px.bar(
    data_frame=hh_copy[mask].groupby(by='Образование')['ЗП (руб)'].median(),
    title='Зависимость медианной ожидаемой зарплаты от уровня образования',
    labels={'value':'Медианная зарплата в рублях', 'Образование':'Уровень образования'}
)
fig.update_layout(title_xanchor='center', title_x=0.5)
fig.update_layout(showlegend=False)
fig.show()

Наибольшая ожидаемая зарплата, что очевидно, наблюдается у соискателей с высшим образованием. \
И у соскателей со средним, и со средним специальным образованием ожидаемая зарплата одинаково низка. Возможно, дело в том, что соискатели со средним образованием раньше начали работать и недостаток в образовании компенсируют опытом работы. \
Уровень образования, конечно же, важен при прогнозировании зарплаты. При переходе со среднего\среднего специального до неоконченного высшего наблюдается рост в 10 тысяч. При последующем переходе до высшего образования снова рост в 10 тысяч 

5. Постройте диаграмму, которая показывает распределение желаемой заработной платы (**"ЗП (руб)"**) в зависимости от города (**"Город"**). Используйте для диаграммы данные о резюме, где желая заработная плата меньше 1 млн рублей.
*Сделайте выводы по полученной диаграмме: как соотносятся медианные уровни желаемой заработной платы и их размах в городах? Как вы считаете, важен ли признак города при прогнозировании заработной платы?*

In [ ]:
mask = hh_copy['ЗП (руб)'] < 1000000
fig = px.box(
    data_frame=hh_copy[mask],
    y='ЗП (руб)',
    x='Город',
    title='Распределение желаемой зарплаты по городам',
    labels={'ЗП (руб)':'Ожидаемая зарплата в рублях'},
    category_orders={'Город':['Москва', 'Санкт-Петербург', 'город-миллионник', 'другие']}
)
fig.update_layout(title_xanchor='center', title_x=0.5)
fig.show()

Видно, что при переходе Москва - Санкт-Петербург - город-миллионник - другие снижается как медианный уровень ожидаемой зарплаты, так и её диапозон. Чем более развит город, тем больше денег там необходимо для поддержания такого же уровня жизни, так же в более развитых городах больше высокооплачиваемых вакансий как в современном производстве, так и в сфере услуг, поэтому высокооплачиваемые специалисты часто перееезжают туда. \
Высокий диапозон зарплат в категории "другие" можно объяснить тем, что туда попали соискатели, которые ищут работу за границей.

6. Постройте **многоуровневую столбчатую диаграмму**, которая показывает зависимость медианной заработной платы (**"ЗП (руб)"**) от признаков **"Готовность к переезду"** и **"Готовность к командировкам"**. Проанализируйте график, сравнив уровень заработной платы в категориях.

In [ ]:
fig = px.bar(
    hh_copy.groupby(by=['Готовность к переезду', 'Готовность к командировкам'], as_index=False)['ЗП (руб)'].median(),
    x='Готовность к переезду',
    y='ЗП (руб)',
    facet_col='Готовность к командировкам',
    labels={'ЗП (руб)':'Желаемая заработная плата в рублях'},
    title="Зависимость заработной платы от готовности к переезду и командировкам"
)
fig.update_layout(title_xanchor='center', title_x=0.5)
fig.show()

Видно, что и готовность к переезду, и готовность к командировкам повышают желаемую зарплату. \
Готовность к командировкам повышает зарплату на 20 тысяч, если соискатель не готов к переезду. И на 15 тысяч, если готов. \
Готовность к переезду повышает зарплату на 1 тысяч, если соискатель не готов к командировкам. И на 5 тысяч, если готов. \
Готовность к командировкам является более важным показателем, чем готовность к переезду. Возможно, это объясняется тем, что професии, подразумевающие командировки, часто бывают представительными и лучше оплачиваются 

7. Постройте сводную таблицу, иллюстрирующую зависимость **медианной** желаемой заработной платы от возраста (**"Возраст"**) и образования (**"Образование"**). На полученной сводной таблице постройте **тепловую карту**. Проанализируйте тепловую карту, сравнив показатели внутри групп.

In [ ]:
fig = px.imshow(
    hh_copy.groupby(by=['Образование', 'Возраст'])['ЗП (руб)'].median().unstack(),
    title='Зависимость зарплаты от возраста и уровня образования',
    labels={'color':'Зарплата'}
)
fig.update_layout(title_xanchor='center', title_x=0.5)
fig.show()

Сразу бросаются в глаза аномалии(высшее образование в 14 лет, среднее специальное в 15 и тд.) \
Видно, что у соискателей с высшим и неоконченным высшим образованием зарплата растет до примерно 40-43 лет, после чего начинает падать. Возможно, дел в том, что к этому возрасту самые квалифицированные специалисты обрастают личными связями в своей сфере, становятся известными внутри неё и меньше нуждаются в поиске работы через специальные сервисы. \
У соискателей со срдним с средним специальным образованием зарплата держится довольно стабильно, без большого роста со временем. 

8. Постройте **диаграмму рассеяния**, показывающую зависимость опыта работы (**"Опыт работы (месяц)"**) от возраста (**"Возраст"**). Опыт работы переведите из месяцев в года, чтобы признаки были в едином масштабе. Постройте на графике дополнительно прямую, проходящую через точки (0, 0) и (100, 100). Данная прямая соответствует значениям, когда опыт работы равен возрасту человека. Точки, лежащие на этой прямой и выше нее - аномалии в наших данных (опыт работы больше либо равен возрасту соискателя)

In [ ]:
fig = px.scatter(
    data_frame=hh_copy,
    x='Возраст',
    y=hh_copy['Опыт работы (месяц)'].apply(lambda x: round(x/12, 2)),
    labels={'y':'Опыт работы'},
    title='Зависимость опыта работы и возраста'
)
fig.add_scatter(
    x=[0, 100],
    y=[0, 100],
    mode='lines',
    line_color='gray',
    showlegend=False
)
fig.update_layout(title_xanchor='center', title_x=0.5)
fig.show()

Сразу видны аномалии, в которых опыт работы превышает возраст.  \
Так же сомнительны случаи, в которых разница опыта работы с возрастом меньше 14 лет

**Дополнительные баллы**

Для получения 2 дополнительных баллов по разведывательному анализу постройте еще два любых содержательных графика или диаграммы, которые помогут проиллюстрировать влияние признаков/взаимосвязь между признаками/распределения признаков. Приведите выводы по ним. Желательно, чтобы в анализе участвовали признаки, которые мы создавали ранее в разделе "Преобразование данных".


In [ ]:
education_mask = (hh_copy['Образование'] == 'среднее') | (hh_copy['Образование'] == 'среднее специальное')

fig = px.box(
    data_frame=hh_copy[education_mask],
    y='Опыт работы (месяц)',
    x='Образование',
    title='Распределение опыта работы в зависимости от уровня образования'
)
fig.update_layout(title_xanchor='center', title_x=0.5)
fig.show()

В блоке с зависимостью медианной зарплаты от уровня образования было сделано предположение о том, что соискатели со средним уровнем образование компенсируют его недостаток(в сравнении со средним специальным) большим опытом. По графику видно, что их опыт действительно больше, что может объяснять одинаковую зарплату 

In [ ]:
fig = px.box(
    data_frame=hh_copy,
    y='Опыт работы (месяц)',
    x='Город',
    title='Распределение опыта работы в зависимости от города',
    category_orders={'Город':['Москва', 'Санкт-Петербург', 'город-миллионник', 'другие']}
)
fig.update_layout(title_xanchor='center', title_x=0.5)
fig.show()

В блоке с распределением ожидаемой зарплаты от города было предположение о том, что более опытные соискатели часто переезжают в более крупные и развитыве города, чем частично объясняется разница в зарплате. \
На графике видно, что наибольший опыт у соискателей в Москве. Это ожидаемо, туда переезжают со всей страны. \
На втором месте города-миллионники. Скорее всего дело в том, что они привлекают профессиональных соискателей со своей области, которые не желают переезжать слишком далеко. \
Санкт-Петербург почти не отличается от городов-миллионников, но все же немного отстает. Видимо, он не является достаточно привлекательным, чтобы в него переезжали специалисты из других областей. 

# Очистка данных

1. Начнем с дубликатов в наших данных. Найдите **полные дубликаты** в таблице с резюме и удалите их. 

In [ ]:
# Удалим полные дубликаты из таблицы
with_dup = hh_copy.shape[0]
hh_copy = hh_copy.drop_duplicates()

# Задание 5.1
print(f'Количесво полных дубликатов = {with_dup-hh_copy.shape[0]}')

2. Займемся пропусками. Выведите информацию **о числе пропусков** в столбцах. 

In [ ]:
# В цикле for пройдем по всем столбцам и проверим на наличие пропусков

for column in hh_copy.columns:
    nan_sum = hh_copy[column].isnull().sum()
    if nan_sum > 0:
        print(f'в признаке {column} содержится {nan_sum} пропуск') 

3. Итак, у нас есть пропуски в 3ех столбцах: **"Опыт работы (месяц)"**, **"Последнее/нынешнее место работы"**, **"Последняя/нынешняя должность"**. Поступим следующим образом: удалите строки, где есть пропуск в столбцах с местом работы и должностью. Пропуски в столбце с опытом работы заполните **медианным** значением.

In [ ]:
# Заполним пропуски в опыте работы медианным значением 
hh_copy['Опыт работы (месяц)'].fillna(hh_copy['Опыт работы (месяц)'].median())

# Удалим записи, в которых есть хотя бы 1 пропуск
hh_copy.dropna(how='any', axis=0, subset=['Последнее/нынешнее место работы', "Опыт работы (месяц)"], thresh=1, inplace=True)

# Задание 5.3
print(f'Результирующее среднее значение опыта работы в месяцах: {round(hh_copy["Опыт работы (месяц)"].mean())}')

4. Мы добрались до ликвидации выбросов. Сначала очистим данные вручную. Удалите резюме, в которых указана заработная плата либо выше 1 млн. рублей, либо ниже 1 тыс. рублей.

In [ ]:
# Задание 5.4
print(f"Количество резюме с желаемой зарплатой меньше 1000 и больше 1000000: {hh_copy[(hh_copy['ЗП (руб)'] > 1000000) | (hh_copy['ЗП (руб)'] < 1000)].shape[0]}")

# Удалим найденные выше выбросы
hh_copy = hh_copy[(hh_copy['ЗП (руб)'] < 1000000) | (hh_copy['ЗП (руб)'] > 1000)]

5. В процессе разведывательного анализа мы обнаружили резюме, в которых **опыт работы в годах превышал возраст соискателя**. Найдите такие резюме и удалите их из данных


In [ ]:
# Задание 5.5
print(f"Количество резюме, в которых опыт работы превышает возраст: {hh_copy[(hh_copy['Возраст'] * 12) < hh_copy['Опыт работы (месяц)']].shape[0]}")

# Удалим найденные выше выбросы
hh_copy = hh_copy[(hh_copy['Возраст'] * 12) > hh_copy['Опыт работы (месяц)']]

6. В результате анализа мы обнаружили потенциальные выбросы в признаке **"Возраст"**. Это оказались резюме людей чересчур преклонного возраста для поиска работы. Попробуйте построить распределение признака в **логарифмическом масштабе**. Добавьте к графику линии, отображающие **среднее и границы интервала метода трех сигм**. Напомним, сделать это можно с помощью метода axvline. Например, для построение линии среднего будет иметь вид:

`histplot.axvline(log_age.mean(), color='k', lw=2)`

В какую сторону асимметрично логарифмическое распределение? Напишите об этом в комментарии к графику.
Найдите выбросы с помощью метода z-отклонения и удалите их из данных, используйте логарифмический масштаб. Давайте сделаем послабление на **1 сигму** (возьмите 4 сигмы) в **правую сторону**.

Выведите таблицу с полученными выбросами и оцените, с каким возрастом соискатели попадают под категорию выбросов?

In [126]:
# Находим аномалии методом z-отклонения
outliers, cleaned = outliers_z_score(data=hh_copy, feature='Возраст', log_scale=True, right=4)

fig = px.histogram(
    data_frame = np.log(hh_copy['Возраст']),
    x=['Возраст'],
    title='Распределение возраста в логарифмическом масштабе'
)

# Ищем среднее значение и стандартное отклонение для границ метода z-отклонения
x = np.log(hh_copy['Возраст'])
mu = x.mean()
sigma = x.std()
lower_bound = mu - 3 * sigma
upper_bound = mu + 4 * sigma


# Добавляем на график линии, соответствующие границам метода z-отклонения и среднему значению признака
fig.add_vline(x=lower_bound)
fig.add_vline(x=upper_bound)
fig.add_vline(x=mu)

fig.update_layout(title_xanchor='center', title_x=0.5)
fig.show()

fig = px.histogram(
    data_frame = outliers,
    x='Возраст',
    title='Распределение аномальных значений возраста'
)
fig.update_layout(title_xanchor='center', title_x=0.5)
fig.show()


# Задание 5.6
print(f'Количество выброс, выявленных методом z-отклонений: {outliers.shape[0]}')

Количество выброс, выявленных методом z-отклонений: 3


Распределение имеет ассиметрию вправо. \
В аномалии попали значения 15 и 100 лет. 